# Using PineconeLocalIndex for Routes

Pinecone Local is an in-memory Pinecone Database emulator available as a Docker image.

It's useful for running tests using the Pinecone local server. Your data will not leave your system, which is also helpful if you want to do testing locally before committing to a Pinecone account.

## Limitations
Pinecone Local has the following limitations:

- Pinecone Local is available only as a Docker image.
- Pinecone Local is an in-memory emulator and is not suitable for production. Records loaded into Pinecone Local do not persist after it is stopped.
- Pinecone Local does not authenticate client requests. API keys are ignored.
- Max number of records per index: 100,000.

## Getting Started
Make sure [Docker](https://docs.docker.com/get-docker/) is installed and running on your local machine.

### Download the latest pinecone-local Docker image:


Download the latest pinecone-local Docker image:

```bash
docker pull ghcr.io/pinecone-io/pinecone-local:latest
```

### Start Pinecone Local:

```bash
docker run -d \
--name pinecone-local \
-e PORT=5080 \
-e PINECONE_HOST=localhost \
-p 5080-6000:5080-6000 \
--platform linux/amd64 \
ghcr.io/pinecone-io/pinecone-local:latest
```



In [16]:
!pip install -qU "semantic-router[pinecone]==0.0.22"

In [1]:
from semantic_router import Route

# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

c:\Users\awesome\anaconda3\envs\py12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
routes

[Route(name='politics', utterances=["isn't politics the best thing ever", "why don't you tell me about your political opinions", "don't you just love the presidentdon't you just hate the president", "they're going to destroy this country!", 'they will save the country!'], description=None, function_schemas=None, llm=None, score_threshold=0.5, metadata={}),
 Route(name='chitchat', utterances=["how's the weather today?", 'how are things going?', 'lovely weather today', 'the weather is horrendous', "let's go to the chippy"], description=None, function_schemas=None, llm=None, score_threshold=0.5, metadata={})]

In [10]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# get at platform.openai.com
os.environ.get("OPENAI_API_KEY") or getpass("Enter OpenAI API key: ")
encoder = OpenAIEncoder(
    name="text-embedding-3-large", score_threshold=0.5, dimensions=1536
)

For Pinecone Local, you can pass the API key as `pclocal`.

In [11]:
import os
from semantic_router.index.pinecone_local import PineconeLocalIndex

os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)
os.environ["PINECONE_API_BASE_URL"] = "http://localhost:5080"

index = PineconeLocalIndex(index_name="route-test", dimensions=1536)

2025-01-31 00:33:41 - semantic_router.utils.logger - WARNING - pinecone_local.py:146 - __init__() - Pinecone API key: pclocal
2025-01-31 00:33:41 - semantic_router.utils.logger - WARNING - pinecone_local.py:147 - __init__() - Pinecone API key os env: pclocal
2025-01-31 00:33:41 - semantic_router.utils.logger - WARNING - pinecone_local.py:169 - __init__() - Default region changed from us-west-2 to us-east-1 in v0.1.0.dev6


In [12]:
from semantic_router.routers import SemanticRouter

router = SemanticRouter(
    encoder=encoder,
    routes=routes,
    index=index,
    auto_sync="local",
)

2025-01-31 00:33:42 - semantic_router.utils.logger - ERROR - pinecone_local.py:271 - _init_index() - index_host exists:http://localhost:5083
2025-01-31 00:33:42 - semantic_router.utils.logger - ERROR - pinecone_local.py:272 - _init_index() - index exists:<pinecone.data.index.Index object at 0x000001B0F1089070>
2025-01-31 00:33:44 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


We can check our route layer and index information.

In [6]:
index.index_host

'http://localhost:5081'

In [7]:
router.index.index_host

'http://localhost:5081'

In [8]:
router.index.client.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'localhost:5081',
              'metric': 'dotproduct',
              'name': 'route-test',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [24]:
router.index.is_ready()

True

In [25]:
router.is_synced()

True

In [26]:
router.get_utterance_diff()

['  chitchat: how are things going?',
 "  chitchat: how's the weather today?",
 "  chitchat: let's go to the chippy",
 '  chitchat: lovely weather today',
 '  chitchat: the weather is horrendous',
 "  politics: don't you just love the presidentdon't you just hate the president",
 "  politics: isn't politics the best thing ever",
 '  politics: they will save the country!',
 "  politics: they're going to destroy this country!",
 "  politics: why don't you tell me about your political opinions"]

In [27]:
router("don't you love politics?")

2025-01-29 23:25:18 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-29 23:25:18 - semantic_router.utils.logger - ERROR - pinecone_local.py:581 - query() - retrying query with vector as str
2025-01-29 23:25:18 - semantic_router.utils.logger - INFO - pinecone_local.py:593 - query() - route_names:['politics', 'politics', 'politics', 'chitchat', 'politics']


RouteChoice(name='politics', function_call=None, similarity_score=None)

In [28]:
router.list_route_names()

['politics', 'chitchat']

In [29]:
router("how are things going?")

2025-01-29 23:25:27 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-29 23:25:27 - semantic_router.utils.logger - ERROR - pinecone_local.py:581 - query() - retrying query with vector as str
2025-01-29 23:25:27 - semantic_router.utils.logger - INFO - pinecone_local.py:593 - query() - route_names:['chitchat', 'chitchat', 'chitchat', 'chitchat', 'chitchat']


RouteChoice(name='chitchat', function_call=None, similarity_score=None)

In [30]:
len(router.index)

10

We can also view all of the records for a given route:

In [31]:
router.index._get_route_ids(route_name="politics")

['politics#64069085d9d6e98e5a80915f69fabe82bac6c742f801bc305c5001dce88f0d19',
 'politics#af8b76111f260cf44fb34f04fcf82927dcbe08e8f47c30f4d571379c1512fac8',
 'politics#d1bb40236c3d95b9c695bfa86b314b6da4eb87e136699563fccae47fccea23e2',
 'politics#ed0f3dd7bd5dea12e55b1953bcd2c562a5ab19f501f6d5ff8c8927652c3904b8',
 'politics#fc6d15f9e6075e6de82b3fbef6722b64353e4eadc8d663b7312a4ed60c43e6f6']

And query:

In [32]:
router("don't you love politics?").name

2025-01-29 23:25:42 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-29 23:25:42 - semantic_router.utils.logger - ERROR - pinecone_local.py:581 - query() - retrying query with vector as str
2025-01-29 23:25:42 - semantic_router.utils.logger - INFO - pinecone_local.py:593 - query() - route_names:['politics', 'politics', 'politics', 'chitchat', 'politics']


'politics'

In [33]:
router("how's the weather today?").name

2025-01-29 23:25:43 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-29 23:25:43 - semantic_router.utils.logger - ERROR - pinecone_local.py:581 - query() - retrying query with vector as str
2025-01-29 23:25:43 - semantic_router.utils.logger - INFO - pinecone_local.py:593 - query() - route_names:['chitchat', 'chitchat', 'chitchat', 'chitchat', 'chitchat']


'chitchat'

In [34]:
router("I'm interested in learning about llama 2").name

2025-01-29 23:25:44 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-29 23:25:44 - semantic_router.utils.logger - ERROR - pinecone_local.py:581 - query() - retrying query with vector as str
2025-01-29 23:25:44 - semantic_router.utils.logger - INFO - pinecone_local.py:593 - query() - route_names:['chitchat', 'politics', 'chitchat', 'politics', 'chitchat']


We can delete or update routes.

In [35]:
len(router.index)

10

## Deleting a Route from the Semantic Router
In this section, we demonstrate how to delete a specific route from the `SemanticRouter` instance. This is useful when you want to remove a route that is no longer needed or to update the routing logic dynamically.


In [36]:
import time

router.delete(route_name="politics")
time.sleep(1)
len(router.index)

2025-01-29 23:27:01 - semantic_router.utils.logger - WARNING - pinecone_local.py:628 - _read_config() - Configuration for sr_lock parameter not found in index.
2025-01-29 23:27:01 - semantic_router.utils.logger - INFO - pinecone_local.py:496 - delete() - Deleted 5 vectors from index route-test.


index is not None, deleting...


5

In [38]:
router("don't you love politics??").name

2025-01-29 23:27:31 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-29 23:27:31 - semantic_router.utils.logger - ERROR - pinecone_local.py:581 - query() - retrying query with vector as str
2025-01-29 23:27:31 - semantic_router.utils.logger - INFO - pinecone_local.py:593 - query() - route_names:['chitchat', 'chitchat', 'chitchat', 'chitchat', 'chitchat']


In [39]:
router.index.get_routes()

[Route(name='chitchat', utterances=['how are things going?', "how's the weather today?", 'the weather is horrendous', 'lovely weather today', "let's go to the chippy"], description=None, function_schemas=None, llm=None, score_threshold=None, metadata={})]

## Asynchronous Route Query with Semantic Router

In this section, we explore how to perform an asynchronous query using the `SemanticRouter`. This approach is beneficial when you want to handle multiple queries concurrently without blocking the execution of your program.

In [41]:
await router.acall("how are things going?")

2025-01-29 23:27:42 - httpx - INFO - _client.py:1729 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-29 23:27:42 - semantic_router.utils.logger - ERROR - pinecone_local.py:796 - _async_query() - current host: http://localhost:5081


RouteChoice(name='chitchat', function_call=None, similarity_score=None)

In [42]:
await router.index._async_list_indexes()

{'indexes': [{'name': 'route-test',
   'dimension': 1536,
   'metric': 'dotproduct',
   'host': 'localhost:5081',
   'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
   'status': {'ready': True, 'state': 'Ready'}}]}

### Stop and remove Pinecone Local

To stop and remove the resources for Pinecone Local, run the following command:

```bash
docker compose down
docker stop pinecone-local
docker rm pinecone-local
```